<a href="https://colab.research.google.com/github/valmirf/redes_neurais_pos/blob/main/RNN/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RNN (Recurrent Neural Network)
Notebook baseado em: https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/.

As redes neurais tradicionais de feed-forward recebem uma quantidade fixa de dados de entrada ao mesmo tempo e produzem uma quantidade fixa de saída a cada vez. Por outro lado, os RNNs não consomem todos os dados de entrada de uma vez. Em vez disso, eles os pegam um de cada vez e em sequência. Em cada etapa, o RNN faz uma série de cálculos antes de produzir uma saída. A saída, conhecida como estado oculto, é então combinada com a próxima entrada na sequência para produzir outra saída. Este processo continua até que o modelo seja programado para terminar ou a sequência de entrada termine.

![](https://raw.githubusercontent.com/valmirf/redes_neurais_esp/main/RNN/FIG/rnn.gif)


Os cálculos em cada etapa de tempo consideram o contexto das etapas de tempo anteriores na forma do estado oculto. Ser capaz de usar essas informações contextuais de entradas anteriores é a essência do sucesso dos RNNs em problemas sequenciais.

Embora possa parecer que uma célula RNN diferente está sendo usada a cada passo de tempo nos gráficos, o princípio subjacente das Redes Neurais Recorrentes é que a célula RNN é na verdade exatamente a mesma e reutilizada por completo.

É possível controlar quando a RNN vai gerar uma saída. Podemos gerar uma saída a cada apresentação de entrada, ou gerar uma saída apenas após apresentar uma entrada completa ou do modo que quisermos. Na Figura abaixo, pode-se ver uma RNN com todas as entradas e saídas. Se quisermos uma saída apenas no final, é só ignorar as entradas inermediárias e gerar apenas a saída final (_Output X_). Como esta saída final já passou por cálculos em todas as células anteriores, o contexto de todas as entradas anteriores foi capturado. Isso significa que o resultado final é realmente dependente de todos os cálculos e entradas anteriores.

![](https://raw.githubusercontent.com/valmirf/redes_neurais_esp/main/RNN/FIG/rnn2.jpg)

##Camadas Intermediárias
Para o caso em que você só precisa de uma única saída de todo o processo, obter essa saída pode ser bastante simples, pois você pode facilmente pegar a saída produzida pela última célula RNN na sequência. Como esta saída final já passou por cálculos em todas as células anteriores, o contexto de todas as entradas anteriores foi capturado. Isso significa que o resultado final é realmente dependente de todos os cálculos e entradas anteriores.

$hidden_t=F(hidden_{t−1},input_t)$

Na primeira etapa, um estado oculto normalmente será semeado como uma matriz de zeros, de modo que possa ser alimentado na célula RNN junto com a primeira entrada na sequência. Nos RNNs mais simples, o estado oculto e os dados de entrada serão multiplicados por matrizes de peso inicializadas por meio de um esquema como Xavier ou Kaiming (você pode ler mais sobre este tópico aqui). O resultado dessas multiplicações será então passado por uma função de ativação (como uma função tanh) para introduzir a não linearidade.

$hidden_t=tanh(weight_{hidden}∗hidden_{t−1}+weight_{input}∗input_t)$

Além disso, se precisarmos de uma saída ao final de cada etapa de tempo, podemos passar o estado oculto que acabamos de produzir por meio de uma camada linear ou apenas multiplicá-lo por outra matriz de peso para obter a forma desejada do resultado.


$output_t=weight_{output}∗hidden_t$

O estado oculto que acabamos de produzir será então realimentado na célula RNN junto com a próxima entrada e este processo continua até que fiquemos sem entrada ou o modelo seja programado para parar de produzir saídas.

##Treinamento com Back-propagation
Semelhante a outras formas de redes neurais, os modelos RNN precisam ser treinados para produzir saídas precisas e desejadas após a passagem de um conjunto de entradas.

Durante o treinamento, para cada dado de treinamento, teremos um rótulo de verdade do terreno correspondente ou simplesmente colocaremos uma "resposta correta" que queremos que o modelo produza. Claro, nas primeiras vezes que passamos os dados de entrada pelo modelo, não obteremos saídas iguais a essas respostas corretas. No entanto, depois de receber esses resultados, o que faremos durante o treinamento é calcular a perda desse processo, que mede o quão longe o resultado do modelo está da resposta correta. Usando essa perda, podemos calcular o gradiente da função de perda para retropropagação.

Com o gradiente que acabamos de obter, podemos atualizar os pesos no modelo de forma adequada para que cálculos futuros com os dados de entrada produzam resultados mais precisos. O peso aqui se refere às matrizes de peso que são multiplicadas com os dados de entrada e estados ocultos durante o passe para frente. Todo esse processo de cálculo dos gradientes e atualização dos pesos é chamado de retropropagação. Combinado com a passagem para a frente, a retropropagação é repetida continuamente, permitindo que o modelo se torne mais preciso com suas saídas a cada vez que os valores das matrizes de peso são modificados para selecionar os padrões dos dados.

Embora possa parecer que cada célula RNN está usando um peso diferente, conforme mostrado nos gráficos, todos os pesos são na verdade os mesmos, pois a célula RNN está sendo essencialmente reutilizada ao longo do processo. Portanto, apenas os dados de entrada e o estado oculto transportado são exclusivos em cada etapa de tempo.

##Implementação
Nesta implementação, será construído um modelo que pode completar sua frase com base em alguns caracteres ou uma palavra usada como entrada.

O modelo será alimentado com uma palavra e preverá qual será o próximo caractere da frase. Este processo se repetirá até que geremos uma frase do tamanho desejado.

Para manter isso curto e simples, não usaremos nenhum conjunto de dados grande ou externo. Em vez disso, definiremos apenas algumas frases para ver como o modelo aprende com essas frases. O processo que essa implementação levará é o seguinte:



1.   Criação do dicionário
2.   Pré-processamento: padding e divisão entre entrada e rótulo
3. Codificação _One-Hot_
4. Definição do Modelo
5. Treinamento do Modelo
6. Avaliação do Modelo





In [ ]:
import torch
from torch import nn

import numpy as np

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Primeiro, definiremos as sentenças que queremos que nosso modelo produza quando alimentado com a primeira palavra ou os primeiros caracteres.

Então, vamos criar um dicionário de todos os caracteres que temos nas frases e mapeá-los para um número inteiro. Isso nos permitirá converter nossos caracteres de entrada em seus respectivos inteiros (`char2int`) e vice-versa (`int2char`).

In [ ]:
text = ['oi como voce esta', 'eu estou bem', 'tenha um bom dia']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

print(char2int)

{'s': 0, 'n': 1, 'v': 2, 'i': 3, 'b': 4, ' ': 5, 'e': 6, 't': 7, 'h': 8, 'm': 9, 'a': 10, 'd': 11, 'c': 12, 'u': 13, 'o': 14}


A seguir, as sentenças de entrada serão preenchidas para garantir que todas as sentenças tenham o mesmo tamanho. Embora os RNNs possam receber entradas de tamanhos variáveis, geralmente queremos alimentar os dados de treinamento em lotes para acelerar o processo de treinamento. Para usar os lotes para treinar em nossos dados, precisaremos garantir que cada sequência nos dados de entrada tenha o mesmo tamanho.

Portanto, na maioria dos casos, o preenchimento pode ser feito preenchendo as sequências muito curtas com valores 0 e corte das sequências muito longas. Em nosso caso, encontraremos o comprimento da sequência mais longa e preencheremos o restante das frases com espaços em branco para corresponder a esse comprimento.

In [ ]:
maxlen = len(max(text, key=len))
print("The longest string has {} characters".format(maxlen))


# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of the sentence matches
# the length of the longest sentence
for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' '

The longest string has 17 characters


Como vamos prever o próximo caractere na sequência, a cada passo de tempo, teremos que dividir cada frase em:

* Dados de entrada:
O último caractere de entrada deve ser excluído, pois não precisa ser alimentado no modelo
*Rótulo/alvo:
Um caractere à frente dos dados de entrada, pois esta será a "resposta correta".

In [ ]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])
    
    # Remove firsts character for target sequence
    target_seq.append(text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: oi como voce est
Target Sequence: i como voce esta
Input Sequence: eu estou bem    
Target Sequence: u estou bem     
Input Sequence: tenha um bom dia
Target Sequence: enha um bom dia 


Agora podemos converter nossas sequências de entrada e destino em sequências de inteiros em vez de caracteres, mapeando-os usando os dicionários que criamos acima. Isso nos permitirá codificar a quente nossa sequência de entrada posteriormente.

In [ ]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

Antes de codificar nossa sequência de entrada em vetores one-hot, definiremos 3 variáveis ​​principais:

* `dict_size`: o número de caracteres únicos que temos em nosso texto. Isso determinará o tamanho de um vetor quente, pois cada caractere terá um índice atribuído nesse vetor

* `seq_len`: o comprimento das sequências que estamos alimentando no modelo. Como padronizamos o comprimento de todas as nossas frases para serem iguais às frases mais longas, este valor será o comprimento máximo - 1, pois removemos a entrada do último caractere também

* `batch_size`: o número de sentenças que definimos e que alimentaremos no modelo como um lote


In [ ]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

Input shape: (3, 16, 15) --> (Batch Size, Sequence Length, One-Hot Encoding Size)


Para começar a construir nosso próprio modelo de rede neural, podemos definir uma classe que herda a classe base de PyTorch (`nn.module`) para todos os módulos de rede neural. Depois de fazer isso, podemos começar a definir algumas variáveis ​​e também as camadas para o nosso modelo sob o construtor. Para este modelo, usaremos apenas 1 camada de RNN seguida por uma camada totalmente conectada. A camada totalmente conectada será responsável por converter a saída RNN em nossa forma de saída desejada.

Também teremos que definir a função forward  em `forward()` como um método de classe. A ordem em que a função direta é executada sequencialmente, portanto, teremos que passar as entradas e o estado oculto inicializado com zero através da camada RNN primeiro, antes de passar as saídas RNN para a camada totalmente conectada. Observe que estamos usando as camadas que definimos no construtor.

O último método que temos que definir é o método que chamamos anteriormente para inicializar o estado oculto - `init_hidden()`. Isso basicamente cria um tensor de zeros na forma de nossos estados ocultos.

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

Depois de definir o modelo acima, teremos que instanciar o modelo com os parâmetros relevantes e definir nossos hiperparâmetros também. Os hiperparâmetros que definimos abaixo são:

* `n_epochs`: Number of Epochs -> Refere-se ao número de vezes que nosso modelo percorrerá todo o conjunto de dados de treinamento
* `lr`: Taxa de aprendizagem -> Isso afeta a taxa em que nosso modelo atualiza os pesos nas células cada vez que a retropropogação é feita

Uma taxa de aprendizagem menor significa que o modelo altera os valores do peso com uma magnitude menor
Uma maior taxa de aprendizagem significa que os pesos são atualizados em uma extensão maior para cada etapa
Semelhante a outras redes neurais, temos que definir o otimizador e a função de perda também. Usaremos CrossEntropyLoss, pois o resultado final é basicamente uma tarefa de classificação.

In [ ]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
print(model)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Model(
  (rnn): RNN(15, 12, batch_first=True)
  (fc): Linear(in_features=12, out_features=15, bias=True)
)


Agora podemos começar nosso treinamento! Como temos apenas algumas frases, esse processo de treinamento é muito rápido. No entanto, conforme progredimos, conjuntos de dados maiores e modelos mais profundos significam que os dados de entrada são muito maiores e o número de parâmetros dentro do modelo que temos que calcular é muito maior.

In [ ]:
# Training Run
input_seq = input_seq.to(device)

for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    #input_seq = input_seq.to(device)
    output, hidden = model(input_seq)
    output = output.to(device)
    target_seq = target_seq.to(device)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 2.3268
Epoch: 20/100............. Loss: 2.0647
Epoch: 30/100............. Loss: 1.7181
Epoch: 40/100............. Loss: 1.3383
Epoch: 50/100............. Loss: 1.0026
Epoch: 60/100............. Loss: 0.7197
Epoch: 70/100............. Loss: 0.5005
Epoch: 80/100............. Loss: 0.3463
Epoch: 90/100............. Loss: 0.2427
Epoch: 100/100............. Loss: 0.1841


Vamos testar nosso modelo agora e ver que tipo de saída obteremos. Antes disso, vamos definir alguma função auxiliar para converter a saída do nosso modelo de volta para texto.

In [ ]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character = character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden


def sample(model, out_len, start='oi'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [ ]:
sample(model, 17, 'tenha')


'tenha um bom dia '